In [15]:
#imports

import os
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from collections import Counter

In [16]:
#paths to the training and test data sets.

TRAIN_DIR = os.getcwd()+"\\train-mails1"
TEST_DIR = os.getcwd()+"\\test-mails1"

###### Step1:

this function takes the mails-->convert them into bag of words-->then count the frequency of each words-->store them as key-value pairs-i.e. dictionaries--->select/considers only the top 3000 such pairs(acc to their freq)-->and return them

In [57]:
#function to create a dictionary

def makeDictionary(root_file): #takes the path of the directory as argument
    allwords=[]
    
    emails=[os.path.join(root_file,f) for f in os.listdir(root_file)]
    for mails in emails:
        with open(mails) as mail:
            for line in mail:
                words=line.split()
                allwords+=words
    
    #this function <Counter()> is used to count hashable objects
    #implicitly creates a hash table of an iterable when invoked
    dictionary= Counter(allwords) 
    #print("dictionary=",dictionary)
    
    listtoremove=dictionary.keys()
    #print("listtoremove=",listtoremove)
    
    for item in list(listtoremove):
        if item.isalpha()==False: #checks if all the characters are alphabet or not
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
            
    dictionary=dictionary.most_common(3500)
    #print("after the most common filter ,dictionary=",dictionary)
    
    return dictionary

In [71]:
def extractfeatures(mail_dir):
    files=[os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] #paths of the files-->stored in files
    features_matrix=np.zeros((len(files),3000))                      #initializes the feature_matrix to a matrix of zeroes
    train_labels=np.zeros(len(files))
    
    count=0
    docId=0
    
    for file in files:
        #print("\nfile=",file)
        with open(file) as f:
            #print("\nf=",f)
            for i, line in enumerate(f):
                #print("\ni=",i,"\t line=",line)
                if i==2:
                    words=line.split()
                    for word in words:
                        wordID=0
                        for i,d in enumerate(dictionary):
                            #print("\nd=",d,"i=",i,"d[0]=",d[0])
                            if d[0]==word:
                                wordID=i
                                features_matrix[docId,wordID]=words.count(word)
            train_labels[docId]=0
            filepathTokens=file.split("\\")
            lastToken=filepathTokens[len(filepathTokens)-1]
            if lastToken.startswith("spmsg"):
                train_labels[docId]=1
                count=count+1
            docId=docId+1
        return features_matrix,train_labels

In [72]:
features_matrix, labels = extractfeatures(TRAIN_DIR)

In [73]:
dictionary = makeDictionary(TRAIN_DIR)

In [74]:
test_feature_matrix, test_labels = extractfeatures(TEST_DIR)